In [1]:
# -*- coding: utf-8 -*-
"""Train or test algorithms on MineRLTreechopVectorObf-v0.

- Author: Prabhasa Kalkur
- Contact: prabhasa.94@gmail.com

Config file for algo: --cfg-path
Expert demo for fD algos: --demo-path (system-dependent)
Env name: env_name
WANDB logs: wandb.init (system-dependent)
"""

import argparse
import datetime
import minerl
import os

import gym
import envs
import numpy as np
from envs.wrappers import MineRLObservationWrapper, MineRLActionWrapper, MineRLDiscreteActionWrapper, MineRLDeterministic
from rl_algorithms import build_agent
import rl_algorithms.common.env.utils as env_utils
import rl_algorithms.common.helper_functions as common_utils
from rl_algorithms.utils import Config

from xvfbwrapper import Xvfb # only for ecelbw00202

import wandb
# wandb.config["more"] = "custom"

def parse_args() -> argparse.Namespace:
    # configurations
    parser = argparse.ArgumentParser(description="Pytorch RL algorithms")
    parser.add_argument(
        "--seed", type=int, default=42, help="random seed for reproducibility"
    )
    parser.add_argument(
        "--cfg-path",
        type=str,
        default="./configs/MineRLTreechopVectorObf_v0/dqn.py", # PARAM 1: ALGORITHM
        help="config path",
    )
    parser.add_argument(
        "--test", dest="test", action="store_true", help="test mode (no training)"
    )
    parser.add_argument(
        "--load-from",
        type=str,
        default=None,
        help="load the saved model and optimizer at the beginning",
    )
    parser.add_argument(
        "--off-render", dest="render", action="store_false", help="turn off rendering"
    )
    parser.add_argument(
        "--render-after",
        type=int,
        default=0,
        help="start rendering after the input number of episode",
    )
    parser.add_argument(
        "--log", dest="log", action="store_false", help="turn on logging"
    )
    parser.add_argument(
        "--save-period", type=int, default=3, help="save model period"
    )
    parser.add_argument(
        "--episode-num", type=int, default=30, help="total episode num"
    )
    parser.add_argument(
        "--max-episode-steps", type=int, default=0, help="max episode step"
    )
    parser.add_argument(
        "--interim-test-num",
        type=int,
        default=1,
        help="number of test during training",
    )
    parser.add_argument(
        "--demo-path",
        type=str,
        # PARAM 2: FOR FD ALGOS
        default = "/home/grads/p/prabhasa/MineRL2020/data/MineRLTreechopVectorObf-v0",  # ecelbw00202
        # default="data/MineRLTreechopVectorObf-v0",  # PK laptop
        help="demonstration path for learning from demo",
    )
    parser.add_argument(
        "--integration-test",
        dest="integration_test",
        action="store_true",
        help="indicate integration test",
    )

#     return parser.parse_args()
    return parser.parse_known_args()


def main():
    """Main."""
    args, unknown = parse_args()

    # PARAM 3: INITILAIZE WANDB
    wandb.init(name='dqn_mtc_obf_1', project="lensminerl_treechop_obf", dir='/home/grads/p/prabhasa/MineRL2020/medipixel', group='dry_run', reinit=True, sync_tensorboard=True) # ecelbw00202
    # wandb.init(name='dqn_mtc_obf_1', project="wandb_on_minerl", dir='C:/GitHub/MineRL-NeurIPS-2020', group='dry_run', reinit=True, sync_tensorboard=True) # PK laptop: locally cloned repo
    # wandb.init(name='dqn_mtc_obf_1', project="wandb_on_minerl", dir='C:/MineRL/medipixel', group='dry_run', reinit=True, sync_tensorboard=True) # PK laptop: locally run code
    # wandb.tensorboard.patch(tensorboardX=True, pytorch=True)

    # PARAM 3: env initialization and wrappers
    env_name = "MineRLTreechopVectorObf-v0"
    # env_name = "MineRLObtainDiamondVectorObf-v0"
    env = gym.make(env_name)
    env = MineRLObservationWrapper(env)
    env = MineRLActionWrapper(env)
    env = MineRLDiscreteActionWrapper(env)  # PARAM 4.1: FOR DISCRETE ACTION SPACES (K-MEANS)
    env = MineRLDeterministic(env, args.seed)
    env = env_utils.set_env(env, args)

    # set a random seed
    common_utils.set_random_seed(args.seed, env)

    # run
    NOWTIMES = datetime.datetime.now()
    curr_time = NOWTIMES.strftime("%y%m%d_%H%M%S")

    cfg = Config.fromfile(args.cfg_path)

    # If running integration test, simplify experiment
    if args.integration_test:
        cfg = common_utils.set_cfg_for_intergration_test(cfg)

    # PK: Added np.array to obs_space and changed is_discrete
    cfg.agent.env_info = dict(
        name=env_name,
        observation_space=np.array(env.observation_space),
        action_space=env.action_space,
        is_discrete=True,  # PARAM 4.2: FOR DISCRETE ACTION SPACES
    )
    cfg.agent.log_cfg = dict(agent=cfg.agent.type, curr_time=curr_time)
    build_args = dict(args=args, env=env)
    agent = build_agent(cfg.agent, build_args)

    if not args.test:
        agent.train()
    else:
        agent.test()

    wandb.join()

/home/grads/p/prabhasa/.conda/envs/lensminerl/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/grads/p/prabhasa/.conda/envs/lensminerl/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
if __name__ == "__main__":
    vdisplay = Xvfb(width=1280, height=740, colordepth=16)
    vdisplay.start()
    main()
    vdisplay.stop()

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
0it [00:00, ?it/s]

[INFO] episode 2, episode step: 8000, total step: 16000, total score: 0.000000
epsilon: 0.940610, loss: 3.284729, avg q-value: 0.028332 (spent 0.358096 sec/step)

[INFO] episode 3, episode step: 8000, total step: 24000, total score: 0.000000
epsilon: 0.861410, loss: 3.897083, avg q-value: 0.011291 (spent 0.460615 sec/step)

[INFO] episode 4, episode step: 7082, total step: 31082, total score: 0.000000
epsilon: 0.791298, loss: 3.359669, avg q-value: 0.004145 (spent 0.459518 sec/step)

[INFO] episode 5, episode step: 8000, total step: 39082, total score: 0.000000
epsilon: 0.712098, loss: 2.875367, avg q-value: 0.034729 (spent 0.459015 sec/step)

[INFO] episode 6, episode step: 8000, total step: 47082, total score: 0.000000
epsilon: 0.632898, loss: 2.603057, avg q-value: 0.024256 (spent 0.466887 sec/step)

[INFO] episode 7, episode step: 8000, total step: 55082, total score: 0.000000
epsilon: 0.553698, loss: 2.287432, avg q-value: 0.018249 (spent 0.476490 sec/step)

[INFO] episode 8, epis

KeyboardInterrupt: 